## Setup

In [1]:
# use full window width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os 
##os.chdir('..')
from matplotlib import pyplot as plt
import numpy as np
import virl
import time
import itertools
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from collections import deque

import numpy as np
import sys
import random
from collections import namedtuple
import collections
import copy

# Import the open AI gym
import gym

import tensorflow as tf
import pandas as pd

from gym.envs.toy_text import discrete

import sklearn.pipeline
import sklearn.preprocessing
from sklearn.kernel_approximation import RBFSampler
print(sys.version)
print(tf.__version__)
#from keras import backend as K

C:\Users\ASUS\AppData\Roaming\Python\Python36\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\AppData\Roaming\Python\Python36\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\ASUS\AppData\Roaming\Python\Python36\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
1.15.0


####  Zhouyang Shen:Help function to see the converge

In [3]:
EpisodeStats = namedtuple("Stats",["episode_lengths", "episode_rewards"])

# This is used to compare the episode length over time,which is always 52
def plot_episode_stats(stats, smoothing_window,environment,noisy, noshow=False):

    # Plot the episode reward over time,so we can see when did it converge
    fig1 = plt.figure(figsize=(10,5))
    rewards_smoothed = pd.Series(stats.episode_rewards).rolling(smoothing_window, min_periods=smoothing_window).mean()
    plt.plot(rewards_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Reward (Smoothed)")
    plt.title("Episode Reward over Time (Smoothed over window size {})".format(smoothing_window))
    plt.savefig('graphs/ps_lfa/converge graph/average performance vs number of episodes'+ "for ps_lfa agent" + " " + environment + " " + noisy + " "+ '.png')
    if noshow:
        plt.close(fig1)
    else:
        plt.show(fig1)
    return fig1

#### Zhouyang Shen:Initialze Environment for observation purpose

In [4]:
env = virl.Epidemic(problem_id=0,noisy=False)
env.observation_space.sample()

array([2.37812848, 1.54003528, 0.25362512, 0.07612969])

#### Zhouyang Shen:Draw state samples from the environment  and train the featurizer for function approximation on states

In [5]:
def get_scaler(env):
    observation_examples = np.array([env.observation_space.sample() for x in range(2000000)])
    scaler = sklearn.preprocessing.StandardScaler()
    scaler.fit(observation_examples)
    return scaler,observation_examples

In [6]:
# Feature Preprocessing: we normalize the data to zero mean and unit variance
# We use samples (10000) from the observation space for normalization

def get_featurizer(env,scaler,observation_samples):
    
    # Used to converte a state to a featurizes represenation for fuction approximation.
    # We use RBF kernels with different variances to gather different parts of the space
    featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=0.0, n_components=10)),
        ])
    ## train the featurizer here 
    featurizer.fit(scaler.transform(observation_samples))
    return featurizer


#### Zhouyang Shen:Define the policy estimator (Function approximator)

In [8]:
class FunctionApproximator():
    """
    Policy Function approximator. 
    taking learning rate 
    """
    
    def __init__(self, featurizer,scaler,learning_rate=0.01,scope="policy_estimator"):
        with tf.variable_scope(scope):
            self.scaler = scaler
            self.featurizer = featurizer

            # Allocate the space space for each states
            self.state = tf.placeholder(tf.float32, [10], "state")
            # Allocate the action space for each step
            self.action = tf.placeholder(dtype=tf.int32, name="action")
            # Initialize Goal(sj) for each state
            self.target = tf.placeholder(dtype=tf.float32, name="target")

            # This is a linear estimator take in:state;output:different h-value related to different actions
            self.output = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(self.state, 0),
                num_outputs=env.action_space.n,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer)
            self.outcome = tf.nn.softmax(self.output)
            # this will produce the policy(𝜋(𝑎𝑗|𝑠𝑗,𝜃)) we needed 
            self.action_probs = tf.squeeze(self.outcome)
            # Loss (we take log of it)and train the function with loss = −𝑙𝑜𝑔(𝜋(𝑎𝑗|𝑠𝑗,𝜃))∗𝐺𝑡
            self.loss = -tf.log(tf.gather(self.action_probs, self.action)) * self.target
            # we use Adam optimizer with provided learning rate to train the function
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.training_operation = self.optimizer.minimize(
                self.loss, global_step=tf.contrib.framework.get_global_step())
            
    def featurize_state(self,state):
        """
        Returns the featurized representation for a state.
        """
        scaled = self.scaler.transform([state])
        featurized = self.featurizer.transform(scaled)
        return featurized[0]

    
    def predict(self, state, sess=None):
        # First transform state to feature representation and then predict the 𝜋(𝑎𝑗|𝑠𝑗,𝜃) for state
        sess = sess or tf.get_default_session()
        state = self.featurize_state(state)        
        return sess.run(self.action_probs, { self.state: state })

    def update(self, state, target, action, sess=None):
        # This will update the parameter by doing 𝜃←𝜃+𝛼𝐺𝑡∇𝜋(𝑎𝑗|𝑠𝑗,𝜃)𝜋(𝑎𝑗|𝑠𝑗,𝜃)
        sess = sess or tf.get_default_session()
        feed_dict = { self.state: self.featurize_state(state), self.target: target, self.action: action  }
        _, loss = sess.run([self.training_operation, self.loss], feed_dict)  # Call the custom optimiser which takes a gradient step
        return loss
    def new_episode(self):        
        self.t_episode  = 0.   

#### Zhouyang Shen:Define the policy search algorithm(Reinforce)

In [9]:
def reinforce(env, estimator, num_episodes, discount_factor=1.0):
    """
    REINFORCE Algorithm. Optimizes the policy
    function approximator using policy search.
    
    Args:
        env: OpenAI environment.
        estimator_policy: Policy Function to be optimized         
        num_episodes: Number of episodes to run for
        discount_factor: reward discount factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    
    """

    # Keeps the track of useful statistics
    stats = EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    # save the transition for sampling at end
    Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
    
    for i_episode in range(num_episodes):
        # Reset the environment and pick the fisrst action
        state = env.reset()
        episode = []
        done = False
        # One step in the environment

        for t in itertools.count():
            # Take a step
            action_probs = estimator.predict(state) # calculate 𝜋(𝑎𝑗|𝑠𝑗,𝜃)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs) # make action base on policy
            ## getting to nest state
            next_state, reward, done, _ = env.step(action)
            
            # Keep track of the transition
            episode.append(Transition(
              state=state, action=action, reward=reward, next_state=next_state, done=done))
            
            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            # Print out which step we're on, useful for debugging.
            print("\rStep {} @ Episode {}/{} ({})".format(
                    t, i_episode + 1, num_episodes, stats.episode_rewards[i_episode - 1]), end="")            

            if done:
                break
                
            state = next_state
    
        # Go through the episode, step-by-step and make policy updates
        estimator.new_episode()
        
        for t, transition in enumerate(episode):                 
            
            # The return, G_t, after this timestep; this is the target for the FunctionApproximator
            G_t = sum(discount_factor**i * t.reward for i, t in enumerate(episode[t:]))
           
            # Update our policy estimator
            estimator.update(transition.state,np.array(G_t),transition.action)       
         
    return stats,estimator

#### Zhouyang Shen:Train on the environment and doing fine tuning and predicting on the action to construct visualization

#### First : make the function to show the action content

In [10]:
def show_action(actions):
    outcome = "\n" 
    for i in range(len(actions)):
        outcome += ("week " + str("{:2}".format(i+1)) + " : ")
        if actions[i] == 0:
            outcome += ("%-17s" %("None"))
        if actions[i] == 1:
            outcome += ("%-17s" %("Full Lockdown"))
        if actions[i] == 2:
            outcome += ("%-17s" %("Track & Trace"))
        if actions[i] == 3:
            outcome += ("%-17s" %("Social Distancing"))
        if len(actions) - 1 != i:
            outcome += (" -> ")
        if (i+1) % 4 == 0:
            outcome += "\n"
    return outcome

#### Second : make the function to run one episode with the trained policy estimator 

In [11]:
def run_one_episode(env,policy_estimator,environment,noisy):
    states = []
    rewards = []
    actions = []
    done = False
    s = env.reset()
    states.append(s)
    while not done:
        # make a step
        action_probs = policy_estimator.predict(s) # calculate 𝜋(𝑎𝑗|𝑠𝑗,𝜃) 
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)# make action base on policy
        next_state, reward, done, _ = env.step(action)

        states.append(next_state)
        rewards.append(reward)
        actions.append(action)
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    # This is the place for visualizing the policy
    print("\n")
    print(show_action(actions))
    labels = ['s[0]: susceptibles', 's[1]: infectious', 's[2]: quarantined', 's[3]: recovereds']
    states = np.array(states)
    for i in range(4):
        axes[0].set_title("The states analysis for" + " " + environment + " " + noisy + " " + "for ps_lfa")
        axes[0].plot(states[:,i], label=labels[i]);
        axes[0].set_xlabel('weeks since start of epidemic')
        axes[0].set_ylabel('State s(t)')
        axes[0].legend()
        axes[1].plot(rewards);
        axes[1].set_title('The Reward to each action' + " " + environment + " " + noisy + " " + "for ps_lfa")
        axes[1].set_xlabel('weeks since start of epidemic')
        axes[1].set_ylabel('reward r(t)')
    
    print('total reward', np.sum(rewards))
    plt.savefig('graphs/ps_lfa/state_and_reward/state and reward for'+ " " + environment + " " + noisy + " " + "for ps_lfa" + '.png')
    return actions

#### Third : make the function to train the agent and evaluate on the trained agent by making one episode 

In [12]:
def policy_search_train_and_predict(env,num_episodes,environment,noisy):
    # Instantiate a FunctionApproximator (i.e. the linear function approximator)

    tf.reset_default_graph()

    global_step = tf.Variable(0, name="global_step", trainable=False)
    
    scaler,observation_samples = get_scaler(env)
    featurizer = get_featurizer(env,scaler,observation_samples)
    policy_estimator = FunctionApproximator(featurizer,scaler,learning_rate=0.001)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        # This is the training stage
        # policy may vary. ~600-1000 seemed to work well.
        stats,estimator = reinforce(env, policy_estimator,num_episodes, discount_factor=0.96)
        ## then,we use the optimized function approximator to predict on optimal policy and gain them
        actions = run_one_episode(env,estimator,environment,noisy)
        
        print("\n")
    return stats

#### Fourth : make the function to train the agent in all environment and problems and evaluate on the trained agent by making one episode

In [13]:
#env0 = virl.Epidemic(problem_id=0,noisy=False)
#policy_estimator,stats = policy_search_train(env,1000)

def overall_training_and_evaluation(num_episodes):
    
    '''Train for each of the environments and problems'''
    
    stats_for_problems_nnoise = []
    for i in range (10):
        env = virl.Epidemic(problem_id=i,noisy=False)
        print("This is problem" + " " + str(i) + " " + "without noise for ps_lfa")
        # train on the problem and evaluating by making one episode
        stats0 = policy_search_train_and_predict(env,num_episodes,"problem" + " " + str(i),"without noise")
        stats_for_problems_nnoise.append(stats0)
        
    stats_for_problems_noise = []
    for j in range (10):
        env = virl.Epidemic(problem_id=j,noisy=True)
         #train on the problem and evaluating by making one episode
        print("This is problem" + " " + str(j) + " " + "with noise for ps_lfa")
        stats1 = policy_search_train_and_predict(env,num_episodes,"problem" + " " + str(j),"with noise")
        stats_for_problems_noise.append(stats1)
        

    env = virl.Epidemic(stochastic=True,noisy=False)
    # train on the stochastic environment and evaluating by making one episode
    print("This is stochastic problem " +  " " + "without noise for ps_lfa")
    stats2 = policy_search_train_and_predict(env,num_episodes,"stochastic","without noise")
    
    env = virl.Epidemic(stochastic=True,noisy=True)
    # train on the stochastic environment and evaluating by making one episode
    print("This is stochastic problem " +  " " + "with noise for ps_lfa")
    stats3 = policy_search_train_and_predict(env,num_episodes,"stochastic","with noise")
    
    print("----------------------------------Learning Curves----------------------------------------------")
    return stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3
    
    

#### Zhouyang Shen:Draw the graph to see how the agent learn (Learning curve)

In [14]:
def show_graphs_policy_psla(stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3):
    '''Used for ploting the converging curve for each training problem and environments'''
    
    for i in range(len(stats_for_problems_nnoise)):
        plot_episode_stats(stats_for_problems_nnoise[i],100,"problem" + " " + str(i),"without noise")
        print("This is the convergence graphs for problem" + " " + str(i) + " " + "without noise for ps_lfa")
    for j in range(len(stats_for_problems_noise)):
        plot_episode_stats(stats_for_problems_noise[j],100,"problem" + " " + str(j),"with noise")
        print("This is the convergence graphs for problem" + " " + str(j) + " " + "with noise for ps_lfa")
        
    plot_episode_stats(stats2,100,"stochastic","without noise")
    print("This is the convergence graphs for stochastic environment" +  " " + "without noise for ps_lfa")
    
    plot_episode_stats(stats3,100,"stochastic","with noise")
    print("This is the convergence graphs for stochastic environment" +  " " + "with noise for ps_lfa")


#### Zhouyang Shen : The overall visualization for all of the environment and problems for passing to run_eval.ipynb file

In [15]:
def visualization_for_policySearch_lfa():
    # Train and evaluate as well as drawing state graph and reward graph
    stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3 = overall_training_and_evaluation(2000)
    # Plot the converging curve
    show_graphs_policy_psla(stats_for_problems_nnoise,stats_for_problems_noise,stats2,stats3)

In [ ]:
##visualization_for_policySearch_lfa()